In [35]:
from visual_behavior import database as db
from visual_behavior.visualization.qc import data_loading as dl

from visual_behavior.change_detection.trials import summarize

from multiprocessing import Pool

import warnings
warnings.filterwarnings("ignore")

In [2]:
%widescreen
%standard_imports

imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


In [49]:
pd.set_option('display.max_rows', 500)

In [3]:
summary = db.get_behavior_session_summary()
id_map = pd.DataFrame([
    {
        'mouse_id':mouse_id, 
        'specimen_id':db.get_specimen_id_from_labtracks_id(mouse_id, show_warnings=False)
    } for mouse_id in summary['mouse_id'].unique()
])
summary = summary.merge(id_map, left_on='mouse_id', right_on='mouse_id', how='left')

In [4]:
cache = dl.get_qc_cache()
experiments = cache.get_experiment_table()

In [5]:
production_behavior_sessions = (
    summary[summary['specimen_id']
            .isin(experiments['specimen_id'].unique())]
            .drop(columns=['_id']).sort_values(by=['mouse_id', 'startdatetime'])
            .reset_index(drop=True)
)

In [19]:
uuid = production_behavior_sessions['behavior_session_uuid'].sample().iloc[0]

In [9]:
vb = db.Database('visual_behavior_data')

In [20]:
uuid

'be37ccc5-3a5d-4e36-8faa-19f03bc492b7'

In [37]:
def get_epoch_summary(uuid):
    try:
        vb = db.Database('visual_behavior_data')
        session_trials = pd.DataFrame(list(vb['behavior_data']['trials'].find({'behavior_session_uuid':uuid})))
        epoch_summary = summarize.epoch_level_summary(session_trials,apply_trial_number_limit = False, epoch_length = 10, clip_vals=[0.01,0.99])
        epoch_summary['criterion_negative'] = -1*epoch_summary['criterion']
        vb.close()
        return epoch_summary
    except KeyError:
        return None

In [53]:
def assign_motivation_group(val,ll,ul):
    if val == -np.inf or val == np.inf:
        return 'uncategorized'
    elif val<ll:
        return 'under_motivated'
    elif val>=ll and val<=ul:
        return 'motivated'
    elif val>ul:
        return 'over_motivated'
    else:
        return 'uncategorized'

In [38]:
%%time
with Pool(32) as pool:
    list_of_epoch_summaries = pool.map(get_epoch_summary, production_behavior_sessions['behavior_session_uuid'].values)
epoch_summaries = pd.concat(list_of_epoch_summaries)

CPU times: user 7.9 s, sys: 332 ms, total: 8.23 s
Wall time: 1min 34s


In [54]:
criterion_thresholds = [-1.25, 1.25]
epoch_summaries['motivation_state'] = epoch_summaries['criterion_negative'].map(
    lambda x:assign_motivation_group(
        x,
        criterion_thresholds[0],
        criterion_thresholds[1]
    )
)

In [43]:
epoch_summaries.to_csv

29485

In [56]:
epoch_summaries.query('criterion_negative > 1.25').sample(10)

metrics,mouse_id,behavior_session_uuid,startdatetime,epoch,num_contingent_trials,response_bias,discrim_p,flashwise_lick_probability,earned_water,lick_latency_median,fraction_time_aborted,number_of_go_trials,number_of_catch_trials,hit_rate,false_alarm_rate,d_prime,reward_lick_count,reward_lick_latency,"mean(HR,FA)",criterion,criterion_negative,motivation_state
1,479426,cf9e6e8d-9149-4f4b-8b71-f89c7b12d374,2019-10-17 15:36:54.373,10.0,29.0,0.862069,0.285256,0.199643,0.168,0.450391,0.562347,28.0,1.0,0.857143,0.99,-1.258777,30.666667,0.184186,0.923571,-1.696959,1.696959,over_motivated
3,470784,f72def20-866e-429e-948e-4ec78e8d22c6,2019-08-15 16:39:55.415,30.0,15.0,1.000000,1.000000,0.328103,0.140,0.441995,0.814273,14.0,1.0,0.990000,0.99,0.000000,25.928571,0.123907,0.990000,-2.326348,2.326348,over_motivated
3,500711,915f5731-f6ec-4250-984f-a5607237f909,2020-01-28 18:26:17.944,30.0,15.0,0.800000,0.871799,0.358273,0.090,0.417008,0.802993,12.0,3.0,0.750000,0.99,-1.651858,18.888889,0.166807,0.870000,-1.500419,1.500419,over_motivated
0,408224,97a0ad0c-c773-4092-b87b-67f03e513498,2018-10-05 15:55:53.529,0.0,12.0,0.750000,0.700311,0.345541,0.060,0.483735,0.765229,9.0,3.0,0.666667,0.99,-1.895621,14.000000,0.112208,0.828333,-1.378538,1.378538,over_motivated
0,457841,16dd1790-2a66-4fa6-92f3-bcda591d01dd,2019-08-30 19:53:54.419,0.0,3.0,1.000000,1.000000,0.338284,0.014,0.325244,0.894950,2.0,1.0,0.990000,0.99,0.000000,24.500000,0.147727,0.990000,-2.326348,2.326348,over_motivated
4,468866,3f20cbd3-6fa6-4d3e-a47b-a8ae26f10b35,2019-08-26 21:26:17.839,40.0,18.0,0.666667,0.716005,0.236273,0.077,0.492067,0.747503,17.0,1.0,0.647059,0.99,-1.948956,19.272727,0.263845,0.818529,-1.351870,1.351870,over_motivated
2,462887,9d80ec6b-c19b-4d9b-9ffa-df7b67f6414d,2019-07-19 15:48:11.151,20.0,34.0,0.941176,0.236250,0.174484,0.217,0.408626,0.533835,32.0,2.0,0.937500,0.99,-0.792227,15.193548,0.168411,0.963750,-1.930234,1.930234,over_motivated
2,477202,dbe5a0bd-7f8c-4529-902e-340cc7ef8529,2019-09-18 20:02:06.476,20.0,8.0,1.000000,1.000000,0.293930,0.042,0.433652,0.898160,6.0,2.0,0.990000,0.99,0.000000,20.000000,0.105640,0.990000,-2.326348,2.326348,over_motivated
2,453990,ccba085f-cd07-47d7-b26a-1a5c476fb2fb,2019-08-15 22:16:54.108,20.0,14.0,1.000000,1.000000,0.237942,0.091,0.450407,0.802264,13.0,1.0,0.990000,0.99,0.000000,22.230769,0.147560,0.990000,-2.326348,2.326348,over_motivated
1,429952,25d15365-5850-42ac-96d5-a03a67659f77,2019-02-12 21:11:15.999,10.0,22.0,0.863636,0.278116,0.204710,0.126,0.416975,0.673450,21.0,1.0,0.857143,0.99,-1.258777,15.166667,0.090803,0.923571,-1.696959,1.696959,over_motivated


In [57]:
epoch_summaries.to_csv(
    '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/behavioral_state_characterization/motivation_states_by_epoch.2020.04.09.csv', 
    index=False
)

In [58]:
experiments

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,
957759566,957020350,957032492,1016840676,VisualBehaviorMultiscope,holding,passed,20190927_457841_4imagesB,OPHYS_4_images_B,MESO.1,2019-09-27 08:28:05.415005,858992726,850862430,F,213.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],75,VISp,NaN
960410028,959458018,959681045,1016840676,VisualBehaviorMultiscope,holding,passed,20191002_457841_6imagesB,OPHYS_6_images_B,MESO.1,2019-10-02 08:52:18.346563,858992726,850862430,F,218.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],75,VISp,NaN
953659745,952430817,952554548,1016840676,VisualBehaviorMultiscope,holding,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,858992726,850862430,F,209.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],75,VISp,NaN
958741224,958105827,958234742,1016840676,VisualBehaviorMultiscope,holding,failed,20190930_457841_3imagesA,OPHYS_3_images_A,MESO.1,2019-09-30 08:24:24.860599,858992726,850862430,F,216.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],75,VISp,NaN
960410023,959458018,959681045,1016840680,VisualBehaviorMultiscope,holding,passed,20191002_457841_6imagesB,OPHYS_6_images_B,MESO.1,2019-10-02 08:52:18.346563,858992726,850862430,F,218.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],150,VISp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902487182,901759468,902135840,895259496,VisualBehaviorTask1B,container_qc,passed,20190709_461946_2imagesB,OPHYS_2_images_B_passive,CAM2P.5,2019-07-09 15:42:29.000000,872873989,862671211,F,112.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN
903405627,902777219,902871565,895259496,VisualBehaviorTask1B,container_qc,passed,20190710_461946_4imagesA,OPHYS_4_images_A,CAM2P.5,2019-07-10 17:19:50.000000,872873989,862671211,F,113.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN
904155155,903711544,903842232,895259496,VisualBehaviorTask1B,container_qc,passed,20190711_461946_5imagesApassive,OPHYS_5_images_A_passive,CAM2P.5,2019-07-11 17:07:58.000000,872873989,862671211,F,114.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN
